# 텍스트 정제 text cleaning
regex를 이용해서..

In [1]:
import re
import pandas as pd

import requests
import pandas as pd
import datetime as dt

import copy

from tqdm.notebook import tqdm
import time

In [2]:
df = pd.read_csv('crawling_df.csv')

In [3]:
new_df = copy.deepcopy(df.filter(['taken_at_timestamp' ,'owner.username','edge_media_to_caption.edges',
                    'edge_media_to_parent_comment.edges', 'edge_media_preview_like.count', 'display_url' , 'shortcode'],
                   axis = 1))

In [4]:
new_df.columns = ['timestamp', 'username', 'post_text', 'comment_text', 'likes', 'pic_url', 'shortcode']

In [5]:
##1. 날짜정보 변환

for i in range(len(new_df)) : 
    new_df.loc[i, "timestamp"] = dt.datetime.fromtimestamp(new_df.loc[i, "timestamp"])

In [6]:
new_df.shape

(207, 7)

In [7]:
#2. post_text정제
#지옥의 정규식

pattern1 = '\\\\n'
pattern2 = "\[|\]|\{|\}|'text'|'node'|:|['']|#"
#pattern3 = "#"
pbar = tqdm(total = len(new_df))
for i in range(len(new_df)) : 
    text = copy.deepcopy(new_df.loc[i, 'post_text'])
    
    #1차 
    a1 = re.sub(pattern1, ' ' , text)
    
    #2차
    a2 = re.sub(pattern2, ' ', a1) 
    
    #3차
    a3 = a2.strip()
    
    new_df.loc[i, 'post_text'] = a3
    pbar.update()
    
pbar.close()

In [8]:
new_df.head()

,timestamp,username,post_text,comment_text,likes,pic_url,shortcode
0,2020-05-01 01:19:31,zyhxxn,뒤통수 전문,[],71,https://scontent-ssn1-1.cdninstagram.com/v/t51...,B_nIlwRHcoR
1,2020-05-01 14:48:30,xxung___irene,부산은 완~전~여~름~☀️ . 광안리해수욕장,"[{'node': {'id': '17857030936917325', 'text': ...",23,https://scontent-ssn1-1.cdninstagram.com/v/t51...,B_olK6YHULG
2,2020-05-06 03:09:35,gut_karma,,"[{'node': {'id': '17842799498160152', 'text': ...",11,https://scontent-ssn1-1.cdninstagram.com/v/t51...,B_0NKSuF1Mx
3,2020-05-04 18:44:51,checkinbusan,- 일정 다 접고 바로 집에 가도 여한 없을 것 같은..😅1시간 기다려 담은 올해 ...,"[{'node': {'id': '17854492351954974', 'text': ...",1181,https://scontent-ssn1-1.cdninstagram.com/v/t51...,B_wumnADqf2
4,2020-05-06 07:23:15,rlawldus1004,부산 해운대 westinchosun 수변공원 민락회센터 날씨좋고 바람좋고 친...,"[{'node': {'id': '18103492210086817', 'text': ...",50,https://scontent-ssn1-1.cdninstagram.com/v/t51...,B_0qMLRDtii


In [9]:
for i in range(5) : 
    print(new_df.loc[i, 'post_text'])

뒤통수 전문
부산은 완~전~여~름~☀️ .  광안리해수욕장

- 일정 다 접고 바로 집에 가도 여한 없을 것 같은..😅1시간 기다려 담은 올해 첫 그러한..부산스러워 더 마음에 드는 사진 -  為了等照片中的路人甲經過🚶 魯蛋大叔在那等了1個小時... 但他很滿意, 因為有拍出自己心中的 釜山印象 你們的釜山印象是什麼呢? 🤗 . . ♡微工商時間♡ 疫情讓店內生意大受影響,  意外開啟了小小選物店的副業,  臉書搜尋 『Check in Busan 小小選物店』  首批商品是文青簡約風的   韓國棉被🔥花色好療癒 ꈍ◡ꈍ   陸續會再推出韓國家居or文具商品  有興趣的人可以進臉書社團逛逛喔◡̈ -  This photo is the result of a 1 hour’ stand, awaiting the passing figure and the proper moment. I like it because it represents part of real Busan in my mind. What does Busan mean to you? 🤗
부산  해운대  westinchosun  수변공원  민락회센터 날씨좋고 바람좋고 친구  더좋고~  그림같은 여기는 삶에 쪄든 직딩의 안식처로세~  김양일상  부산여행  힐링  바다  하늘  여유  감성  사진   패션스타그램  ootd  패피  욜로  직장인일탈


In [10]:
#3. comment_text 정제
#지옥의 정규식...

pbar = tqdm(total = len(new_df))

for i in range(len(new_df)) : 
    text = copy.deepcopy(new_df.loc[i, 'comment_text'])
    
    b = text.split(',')
    
    b2 = str()
    
    for j in range(len(b)) : 
        if bool(re.search('text', b[j])) == True : 
            b2 = b2 + b[j]
            
    b3 = re.sub("'text'|'|@[a-z0-9_.]+|:", '', b2)
    b4 = re.sub("#", ' ', b3)

    new_df.loc[i, 'comment_text'] = b4
    
    pbar.update()
    
pbar.close()

In [11]:
for i in range(10) : 
    print(new_df.loc[i, 'comment_text'])


  헐 부산이라뇨? 전 해운대입니다만 👀👏🏻   헛 나 해운대 먼저 갔다가 지금 광안리야ㅋㅋㅋㅋ  갑자기 부산이요???   ㅋㅋㅋㅋㅋㅋㅋ동번서번입니다🤗  완전 한적하네 부러버   사람 바글바글인데 다 지워버렸어...^^!  🌼    부산 광안리 국내여행 부산여행 인스타데일리 일상스타그램 일상
  ✌🏻   부산 해운대 밤바다 부산여행 부산가볼만한곳 셀피 셀카 셀스타그램 오오티디 아웃핏 좋아요 좋반 소통 팔로우 busan selfie ootd instagood fff  최근 게시물 2개 반사요!
  👏    부산여행 대한민국구석구석 국내여행 감성작가 여행지추천 사진에미치다 사진의언어 소니이미지갤러리 나의사진이야기 색감미술관 갬성스타그램 소니미러리스 사진에관하여 부산데이트 호천마을 산복도로 소니미러리스 쌈마이웨이 부산여행추천 부산가볼만한곳\n busan busantravel busantrip koreabyme triptokorea koreatrip 釜山 sonya7m3  이곳이 김지원과 박서준이 드라마 찍었던 동네 아녜요?   쌈마이웨이요!😊   저 계단에서 앞을 보면 바다 보여요?  啊！！我想起三流之路的南日BAR 真的超漂亮    厲害～👍 就是虎泉村喔😊  「釜山人 」都好親切 很容易和陌生人聊天（語言沒有很通但還是可以很開心\n真的很喜歡釜山誒    好喜歡這個答案😊 謝謝分享💛  와우 사장님 날씨도 더운데 고생많으셨겠어요!!   계단 몇번 오르내리다 기절할뻔 뭐하고 지냄??   저야 뭐..놀고먹기 중입니다..  😃💙💚💛🧡❤️🥰👍👍👍    Thank you! 😊  已加😏    哈哈 謝謝注意到廣告😍  人很親切 海很美    簡單一句話，道盡釜山的樣貌👏👏👏  這是草梁故事路嘛！夢幻！    是虎泉村～ 韓劇三流之路的拍攝地點😊   哈哈，謝謝！收入口袋名單V(^_^)V  부산스럽다는 문구가 정말 공감되어요. 정비되는게 좋겠지만   맞아요! 오랫동안 담을수 있었으면 좋겠어요 산복도로가 하나둘 사라져 가고있어 사진들이 더 소중하네요😊    giống trong fight for my way nhể

In [12]:
new_df.to_csv('cleaning완료.csv', encoding = 'utf-8-sig')

## 사진 추출

In [13]:
import os
import urllib.request as req

dir_name = "prod_imgs"
os.makedirs(dir_name, exist_ok = True) #새로 폴더 만들어 + 부드럽게 실행시킬수있는 옵션
photo_name = "name"
pbar = tqdm(total = len(new_df))
for i in range(len(new_df)) : 
    req.urlretrieve(url = new_df.loc[i,'pic_url'], 
                    filename = dir_name + '/' + photo_name + str(i) + ".jpg")
    pbar.update()
pbar.close()

KeyboardInterrupt: 